# Toy Example of traceCB

In this notebook, we will use example data to run traceCB. First, load data from `data/toy_example/`.

## Set up paths

In [1]:
# set path and load data
import os
import pandas as pd

main_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
print("Main Directory set to:", main_dir)

data_dir = os.path.join(main_dir, "data", "toy_example")
print("Data Directory set to:", data_dir)

Main Directory set to: /home/wjiang49/traceCB
Data Directory set to: /home/wjiang49/traceCB/data/toy_example


Check whether the directory has been set up to: `<your_path>/traceCB`, and the data is in `<your_path>/traceCB/data/toy_example/`.

## Load and preprocess data

In [2]:
# Load data
target_gene = "ENSG00000025708"  # Example gene
## summary statistics for EAS, EUR, and tissue of EUR
eas_summary_df = pd.read_csv(os.path.join(data_dir, "eas_summary_statistics.csv"))
eur_summary_df = pd.read_csv(os.path.join(data_dir, "eur_summary_statistics.csv"))
tissue_summary_df = pd.read_csv(os.path.join(data_dir, "tissue_summary_statistics.csv"))
## ld matrix for EAS, EUR, and cross-population
eas_ld_df = pd.read_csv(os.path.join(data_dir, "eas_ld.csv"))
eur_ld_df = pd.read_csv(os.path.join(data_dir, "eur_ld.csv"))
cross_ld_df = pd.read_csv(os.path.join(data_dir, "cross_ld.csv"))
## cell type proportion in the tissue
celltype_proportion_df = pd.read_csv(os.path.join(data_dir, "celltype_proportion.csv"))
celltype_proportion_percentage = celltype_proportion_df.loc[
    celltype_proportion_df["Cell_type"] == "Monocytes", "Proportion"
].iloc[0]
celltype_proportion = celltype_proportion_percentage / 100.0
## calculate a constant used in GMMtissue
other_celltype_proportions = (
    celltype_proportion_df.loc[
        celltype_proportion_df["Cell_type"] != "Monocytes", "Proportion"
    ]
    / 100.0
).to_numpy()

pii_pij_sum = sum(
    [
        i * j
        for i in other_celltype_proportions
        for j in other_celltype_proportions
        if i != j
    ]
)
pi2_omega_sum_const = 2 * celltype_proportion + pii_pij_sum / (1 - celltype_proportion)

In [3]:
print("Head of Summary DataFrames should look like this:\n", eas_summary_df.head())
print("Head of LD DataFrames should look like this:\n", eas_ld_df.head())

Head of Summary DataFrames should look like this:
          RSID             GENE      BETA        SE         Z      PVAL    N
0  rs75674482  ENSG00000025708  0.270058  0.253239  1.066417  0.288728  105
1    rs135845  ENSG00000025708  0.223593  0.141665  1.578320  0.117558  105
2    rs135846  ENSG00000025708  0.224359  0.141582  1.584656  0.116110  105
3   rs8141979  ENSG00000025708  0.156355  0.148230  1.054817  0.293977  105
4  rs35089849  ENSG00000025708  0.248425  0.253238  0.980993  0.328895  105
Head of LD DataFrames should look like this:
    CHR          SNP        BP  ENSG00000025708
0   22    rs2005631  16869565              0.0
1   22   rs35416799  16869887              0.0
2   22  rs188798391  16878941              0.0
3   22  rs186070939  16879285              0.0
4   22  rs141431439  16880117              0.0


In [4]:
## TL;DR: Align all the input data by RSID/SNP
gene_df = (
    eas_summary_df.merge(
        eur_summary_df.drop(columns=["GENE"]),
        on="RSID",
        suffixes=("_tar", "_aux"),
        how="inner",
    )
    .merge(
        tissue_summary_df.drop(columns=["GENE"]).rename(
            columns={
                col: f"{col}_tissue"
                for col in tissue_summary_df.columns
                if col != "RSID"
            }
        ),
        on="RSID",
    )
    .merge(
        eas_ld_df.rename(columns={target_gene: "LD_tar", "SNP": "RSID"}).loc[
            :, ["RSID", "LD_tar"]
        ],
        on="RSID",
        how="inner",
    )
    .merge(
        eur_ld_df.rename(columns={target_gene: "LD_aux", "SNP": "RSID"}).loc[
            :, ["RSID", "LD_aux"]
        ],
        on="RSID",
        how="inner",
    )
    .merge(
        cross_ld_df.rename(columns={target_gene: "LD_x", "SNP": "RSID"}).loc[
            :, ["RSID", "LD_x"]
        ],
        on="RSID",
        how="inner",
    )
)
print("Head of aligned gene DataFrame should looks like:\n", gene_df.head())

Head of aligned gene DataFrame should looks like:
          RSID             GENE  BETA_tar    SE_tar     Z_tar  PVAL_tar  N_tar  \
0  rs75674482  ENSG00000025708  0.270058  0.253239  1.066417  0.288728    105   
1    rs135845  ENSG00000025708  0.223593  0.141665  1.578320  0.117558    105   
2    rs135846  ENSG00000025708  0.224359  0.141582  1.584656  0.116110    105   
3   rs8141979  ENSG00000025708  0.156355  0.148230  1.054817  0.293977    105   
4  rs35089849  ENSG00000025708  0.248425  0.253238  0.980993  0.328895    105   

   BETA_aux    SE_aux     Z_aux  PVAL_aux  N_aux  BETA_tissue  SE_tissue  \
0 -0.127073  0.218847 -0.580648  0.562223    191     0.017989   0.045871   
1 -0.457850  0.185965 -2.462022  0.014779    191     0.031253   0.030403   
2 -0.451186  0.185522 -2.431981  0.016018    191     0.020958   0.028198   
3 -0.151611  0.113418 -1.336745  0.183028    191     0.006633   0.021028   
4  0.112325  0.148428  0.756764  0.450204    191     0.009180   0.031176   

   Z_

## LDSC and GMM (traceC and traceCB)

Then, load necessary libraries from `src`.

In [5]:
import sys

# Add the parent directory to the system path to import modules from 'src'
module_path = os.path.abspath(os.path.join(os.getcwd(), "../src"))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
from traceCB.ldsc import Run_Cross_LDSC
from traceCB.gmm import GMM, GMMtissue
from traceCB.run_gmm import clip_correlation
from traceCB.utils import z2p, MIN_FLOAT

### Step 1a: Calculate heritability and co-heritability using `Run_Cross_LDSC` (cross population LD score regression).

In [6]:
Omega, Omega_se = Run_Cross_LDSC(
    (gene_df.BETA_tar / gene_df.SE_tar).to_numpy(),
    gene_df.N_tar.to_numpy(),
    gene_df.LD_tar.to_numpy(),
    (gene_df.BETA_aux / gene_df.SE_aux).to_numpy(),
    gene_df.N_aux.to_numpy(),
    gene_df.LD_aux.to_numpy(),
    gene_df.LD_x.to_numpy(),
    np.array([1, 1, 0]),  # fixed intercepts
)
Omega_p = z2p(Omega / Omega_se)
### threshold Omega
for i in range(Omega.shape[0]):
    for j in range(Omega.shape[1]):
        if Omega_p[i, j] > 0.10:
            Omega[i, j] = MIN_FLOAT
if np.any(Omega <= MIN_FLOAT):
    print("Co-heritability not significant, please try another gene")
Omega[0, 1], cor_clipped = clip_correlation(Omega[0, 0], Omega[1, 1], Omega[0, 1])
Omega[1, 0] = Omega[0, 1]
Omega_p = z2p(Omega / Omega_se)

### Step 1b: Calculate $\Sigma_O$ using Run_Cross_LDSC with European ct-eQTL and tissue eQTL.

In [7]:
# Calculate Sigma_o
pi2_omega_sum = 0.0
run_gmm_tissue = True  # set default to run GMM with tissue

aux_Omega_matrix, aux_Omega_matrix_se = Run_Cross_LDSC(
    gene_df.BETA_aux / gene_df.SE_aux,
    gene_df.N_aux,
    gene_df.LD_aux,
    gene_df.BETA_tissue / gene_df.SE_tissue,
    gene_df.N_tissue,
    gene_df.LD_aux,
    gene_df.LD_aux,
    np.array([1.0, 1.0, 0.0]),
)

aux_Omega_matrix[0, 1], _ = clip_correlation(
    aux_Omega_matrix[0, 0], aux_Omega_matrix[1, 1], aux_Omega_matrix[0, 1]
)
aux_Omega_matrix[1, 0] = aux_Omega_matrix[0, 1]

if np.all(z2p(aux_Omega_matrix / aux_Omega_matrix_se) < 0.10):
    pi2_omega_sum = (
        aux_Omega_matrix[1, 1]
        - celltype_proportion**2 * Omega[1, 1]
        - pi2_omega_sum_const
        * np.maximum(aux_Omega_matrix[0, 1] - celltype_proportion * Omega[1, 1], 0)
    )
    if pi2_omega_sum < 0:
        run_gmm_tissue = False
else:
    run_gmm_tissue = False

if not run_gmm_tissue:
    print(
        "GMMtissue will not be run due to non-significant co-heritability with tissue or negative SIGMAO. Please try another gene."
    )

Step 2: Run core `GMM` (TraceC) and `GMMTissue` (TraceCB) algorithm.

In [8]:
# Define functions to apply GMM and GMMtissue to each row
def apply_gmm(row):
    return GMM(
        Omega,
        np.eye(2),
        row["BETA_tar"],
        row["SE_tar"],
        row["LD_tar"],
        row["BETA_aux"],
        row["SE_aux"],
        row["LD_aux"],
        row["LD_x"],
    )


def apply_gmm_tissue(row):
    return GMMtissue(
        Omega,
        np.eye(3),
        row["BETA_tar"],
        row["SE_tar"],
        row["LD_tar"],
        row["BETA_aux"],
        row["SE_aux"],
        row["LD_aux"],
        row["LD_x"],
        row["BETA_tissue"],
        row["SE_tissue"],
        pi2_omega_sum,
        celltype_proportion,
    )


# Apply the functions to the DataFrame, or you can use other parallelization methods
gmm_c_results = gene_df.apply(apply_gmm, axis=1, result_type="expand")
gmm_c_results.columns = ["TAR_CBETA", "TAR_CSE", "AUX_CBETA", "AUX_CSE"]

gmm_t_results = gene_df.apply(apply_gmm_tissue, axis=1, result_type="expand")
gmm_t_results.columns = ["TAR_TBETA", "TAR_TSE", "AUX_TBETA", "AUX_TSE"]

# Combine original and new results into a single DataFrame
gmm_results = pd.concat(
    [
        gene_df[
            [
                "RSID",
                "BETA_tar",
                "SE_tar",
                "Z_tar",
                "PVAL_tar",
                "BETA_aux",
                "SE_aux",
                "Z_aux",
                "PVAL_aux",
                "BETA_tissue",
                "SE_tissue",
                "Z_tissue",
                "PVAL_tissue",
            ]
        ].copy(),
        gmm_c_results,
        gmm_t_results,
    ],
    axis=1,
)

# Rename columns to match the original format
gmm_results.rename(
    columns={
        "BETA_tar": "TAR_SBETA",
        "SE_tar": "TAR_SSE",
        "Z_tar": "TAR_SZ",
        "PVAL_tar": "TAR_SPVAL",
        "BETA_aux": "AUX_SBETA",
        "SE_aux": "AUX_SSE",
        "Z_aux": "AUX_SZ",
        "PVAL_aux": "AUX_SPVAL",
    },
    inplace=True,
)

# Calculate Z-scores and P-values for the new GMM results
for model in ["C", "T"]:  # C for cross, T for tissue
    for pop in ["TAR", "AUX"]:
        beta_col = f"{pop}_{model}BETA"
        se_col = f"{pop}_{model}SE"
        z_col = f"{pop}_{model}Z"
        pval_col = f"{pop}_{model}PVAL"

        gmm_results.loc[:, z_col] = gmm_results[beta_col] / gmm_results[se_col]
        gmm_results.loc[:, pval_col] = z2p(gmm_results[z_col])

# Reorder columns
final_columns = [
    "RSID",
    "TAR_SBETA",
    "TAR_CBETA",
    "TAR_TBETA",
    "TAR_SSE",
    "TAR_CSE",
    "TAR_TSE",
    "TAR_SZ",
    "TAR_CZ",
    "TAR_TZ",
    "TAR_SPVAL",
    "TAR_CPVAL",
    "TAR_TPVAL",
    "AUX_SBETA",
    "AUX_CBETA",
    "AUX_TBETA",
    "AUX_SSE",
    "AUX_CSE",
    "AUX_TSE",
    "AUX_SZ",
    "AUX_CZ",
    "AUX_TZ",
    "AUX_SPVAL",
    "AUX_CPVAL",
    "AUX_TPVAL",
    "BETA_tissue",
    "SE_tissue",
    "Z_tissue",
    "PVAL_tissue",
]
gmm_results = gmm_results[final_columns]

pd.set_option("display.max_columns", 30)
print("Head of GMM results DataFrame should look like:\n", gmm_results.head())

Head of GMM results DataFrame should look like:
          RSID  TAR_SBETA  TAR_CBETA  TAR_TBETA   TAR_SSE   TAR_CSE   TAR_TSE  \
0  rs75674482   0.270058   0.185148   0.193276  0.253239  0.235911  0.230797   
1    rs135845   0.223593   0.236687   0.235555  0.141665  0.141524  0.141511   
2    rs135846   0.224359   0.237014   0.236139  0.141582  0.141446  0.141434   
3   rs8141979   0.156355   0.136936   0.137520  0.148230  0.147208  0.147179   
4  rs35089849   0.248425   0.205571   0.200733  0.253238  0.176277  0.173052   

     TAR_SZ    TAR_CZ    TAR_TZ  TAR_SPVAL  TAR_CPVAL  TAR_TPVAL  AUX_SBETA  \
0  1.066417  0.784823  0.837427   0.288728   0.432557   0.402352  -0.127073   
1  1.578320  1.672418  1.664561   0.117558   0.094442   0.096000  -0.457850   
2  1.584656  1.675657  1.669608   0.116110   0.093805   0.094997  -0.451186   
3  1.054817  0.930220  0.934374   0.293977   0.352257   0.350111  -0.151611   
4  0.980993  1.166183  1.159954   0.328895   0.243541   0.246068   0.112325

In [9]:
print("The columns of the GMM results DataFrame are:\n", gmm_results.columns.tolist())

The columns of the GMM results DataFrame are:
 ['RSID', 'TAR_SBETA', 'TAR_CBETA', 'TAR_TBETA', 'TAR_SSE', 'TAR_CSE', 'TAR_TSE', 'TAR_SZ', 'TAR_CZ', 'TAR_TZ', 'TAR_SPVAL', 'TAR_CPVAL', 'TAR_TPVAL', 'AUX_SBETA', 'AUX_CBETA', 'AUX_TBETA', 'AUX_SSE', 'AUX_CSE', 'AUX_TSE', 'AUX_SZ', 'AUX_CZ', 'AUX_TZ', 'AUX_SPVAL', 'AUX_CPVAL', 'AUX_TPVAL', 'BETA_tissue', 'SE_tissue', 'Z_tissue', 'PVAL_tissue']


where 'TAR_\*' means the target population (here is EAS), 'AUX_\*' means the auxiliary population (here is EUR), and '\*_S\*', '\*_C\*', '\*_T\*' means the summary statistics, cross-population enhancement, and cross-population with tissue enhancement results, respectively.

If you would like to calculate the effect sample size of each result, you can use the `calculate_Neff` function from `src.traceCB.run_gmm`:

In [10]:
from traceCB.run_gmm import calculate_Neff

## calculate neff of summary statistics in eas
tar_sneff = calculate_Neff(
    gmm_results.TAR_SZ,
    Omega[0, 0],
    gene_df.LD_tar,
)
## calculate neff of traceC enhancement in eas
tar_cneff = calculate_Neff(
    gmm_results.TAR_CZ,
    Omega[0, 0],
    gene_df.LD_tar,
)
## calculate neff of traceCB enhancement in eas
tar_tneff = calculate_Neff(
    gmm_results.TAR_TZ,
    Omega[0, 0],
    gene_df.LD_tar,
)
print(
    f"Effect sample size of:\nsummary statistics: {tar_sneff},\ntraceC enhancement: {tar_cneff},\ntraceCB enhancement: {tar_tneff}"
)

Effect sample size of:
summary statistics: 68.1234527359333,
traceC enhancement: 98.44082852444448,
traceCB enhancement: 107.30914032841594


That's it! You can now use traceCB to analyze your data.